In [1]:
import warnings
warnings.filterwarnings('ignore')

import psycopg2
from sqlalchemy import create_engine, Table, Column, MetaData, DateTime, Float, Integer, String, ARRAY, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref, sessionmaker, joinedload
from datetime import datetime
import pandas as pd
from rake_nltk import Rake
import numpy as np
import re, string
import nltk
from sentence_transformers import SentenceTransformer
from keras.preprocessing.text import text_to_word_sequence
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

stopwords = set(stopwords.words('english'))
extraWords= ['whence', 'here', 'show', 'were', 'why',"'s'","n't","'ve", 'n’t', 'the', 'whereupon', 'not', 'more', 'how', 'eight', 'indeed', 'i', 'only', 'via', 'nine', 're', 'themselves', 'almost', 'to', 'already', 'front', 'least', 'becomes', 'thereby', 'doing', 'her', 'together', 'be', 'often', 'then', 'quite', 'less', 'many', 'they', 'ourselves', 'take', 'its', 'yours', 'each', 'would', 'may', 'namely', 'do', 'whose', 'whether', 'side', 'both', 'what', 'between', 'toward', 'our', 'whereby', "'m", 'formerly', 'myself', 'had', 'really', 'call', 'keep', "'re", 'hereupon', 'can', 'their', 'eleven', '’m', 'even', 'around', 'twenty', 'mostly', 'did', 'at', 'an', 'seems', 'serious', 'against', "n't", 'except', 'has', 'five', 'he', 'last', '‘ve', 'because', 'we', 'himself', 'yet', 'something', 'somehow', '‘m', 'towards', 'his', 'six', 'anywhere', 'us', '‘d', 'thru', 'thus', 'which', 'everything', 'become', 'herein', 'one', 'in', 'although', 'sometime', 'give', 'cannot', 'besides', 'across', 'noone', 'ever', 'that', 'over', 'among', 'during', 'however', 'when', 'sometimes', 'still', 'seemed', 'get', "'ve", 'him', 'with', 'part', 'beyond', 'everyone', 'same', 'this', 'latterly', 'no', 'regarding', 'elsewhere', 'others', 'moreover', 'else', 'back', 'alone', 'somewhere', 'are', 'will', 'beforehand', 'ten', 'very', 'most', 'three', 'former', '’re', 'otherwise', 'several', 'also', 'whatever', 'am', 'becoming', 'beside', '’s', 'nothing', 'some', 'since', 'thence', 'anyway', 'out', 'up', 'well', 'it', 'various', 'four', 'top', '‘s', 'than', 'under', 'might', 'could', 'by', 'too', 'and', 'whom', '‘ll', 'say', 'therefore', "'s", 'other', 'throughout', 'became', 'your', 'put', 'per', "'ll", 'fifteen', 'must', 'before', 'whenever', 'anyone', 'without', 'does', 'was', 'where', 'thereafter', "'d", 'another', 'yourselves', 'n‘t', 'see', 'go', 'wherever', 'just', 'seeming', 'hence', 'full', 'whereafter', 'bottom', 'whole', 'own', 'empty', 'due', 'behind', 'while', 'onto', 'wherein', 'off', 'again', 'a', 'two', 'above', 'therein', 'sixty', 'those', 'whereas', 'using', 'latter', 'used', 'my', 'herself', 'hers', 'or', 'neither', 'forty', 'thereupon', 'now', 'after', 'yourself', 'whither', 'rather', 'once', 'from', 'until', 'anything', 'few', 'into', 'such', 'being', 'make', 'mine', 'please', 'along', 'hundred', 'should', 'below', 'third', 'unless', 'upon', 'perhaps', 'ours', 'but', 'never', 'whoever', 'fifty', 'any', 'all', 'nobody', 'there', 'have', 'anyhow', 'of', 'seem', 'down', 'is', 'every', '’ll', 'much', 'none', 'further', 'me', 'who', 'nevertheless', 'about', 'everywhere', 'name', 'enough', '’d', 'next', 'meanwhile', 'though', 'through', 'on', 'first', 'been', 'hereby', 'if', 'move', 'so', 'either', 'amongst', 'for', 'twelve', 'nor', 'she', 'always', 'these', 'as', '’ve', 'amount', '‘re', 'someone', 'afterwards', 'you', 'nowhere', 'itself', 'done', 'hereafter', 'within', 'made', 'ca', 'them', 'her', 'during', 'among', 'thereafter', 'only', 'hers', 'in', 'none', 'with', 'un', 'put', 'hence', 'each', 'would', 'have', 'to', 'itself', 'that', 'seeming', 'hereupon', 'someone', 'eight', 'she', 'forty', 'much', 'throughout', 'less', 'was', 'interest', 'elsewhere', 'already', 'whatever', 'or', 'seem', 'fire', 'however', 'keep', 'detail', 'both', 'yourselves', 'indeed', 'enough', 'too', 'us', 'wherein', 'himself', 'behind', 'everything', 'part', 'made', 'thereupon', 'for', 'nor', 'before', 'front', 'sincere', 'really', 'than', 'alone', 'doing', 'amongst', 'across', 'him', 'another', 'some', 'whoever', 'four', 'other', 'latterly', 'off', 'sometime', 'above', 'often', 'herein', 'am', 'whereby', 'although', 'who', 'should', 'amount', 'anyway', 'else', 'upon', 'this', 'when', 'we', 'few', 'anywhere', 'will', 'though', 'being', 'fill', 'used', 'full', 'thru', 'call', 'whereafter', 'various', 'has', 'same', 'former', 'whereas', 'what', 'had', 'mostly', 'onto', 'go', 'could', 'yourself', 'meanwhile', 'beyond', 'beside', 'ours', 'side', 'our', 'five', 'nobody', 'herself', 'is', 'ever', 'they', 'here', 'eleven', 'fifty', 'therefore', 'nothing', 'not', 'mill', 'without', 'whence', 'get', 'whither', 'then', 'no', 'own', 'many', 'anything', 'etc', 'make', 'from', 'against', 'ltd', 'next', 'afterwards', 'unless', 'while', 'thin', 'beforehand', 'by', 'amoungst', 'you', 'third', 'as', 'those', 'done', 'becoming', 'say', 'either', 'doesn', 'twenty', 'his', 'yet', 'latter', 'somehow', 'are', 'these', 'mine', 'under', 'take', 'whose', 'others', 'over', 'perhaps', 'thence', 'does', 'where', 'two', 'always', 'your', 'wherever', 'became', 'which', 'about', 'but', 'towards', 'still', 'rather', 'quite', 'whether', 'somewhere', 'might', 'do', 'bottom', 'until', 'km', 'yours', 'serious', 'find', 'please', 'hasnt', 'otherwise', 'six', 'toward', 'sometimes', 'of', 'fifteen', 'eg', 'just', 'a', 'me', 'describe', 'why', 'an', 'and', 'may', 'within', 'kg', 'con', 're', 'nevertheless', 'through', 'very', 'anyhow', 'down', 'nowhere', 'now', 'it', 'cant', 'de', 'move', 'hereby', 'how', 'found', 'whom', 'were', 'together', 'again', 'moreover', 'first', 'never', 'below', 'between', 'computer', 'ten', 'into', 'see', 'everywhere', 'there', 'neither', 'every', 'couldnt', 'up', 'several', 'the', 'i', 'becomes', 'don', 'ie', 'been', 'whereupon', 'seemed', 'most', 'noone', 'whole', 'must', 'cannot', 'per', 'my', 'thereby', 'so', 'he', 'name', 'co', 'its', 'everyone', 'if', 'become', 'thick', 'thus', 'regarding', 'didn', 'give', 'all', 'show', 'any', 'using', 'on', 'further', 'around', 'back', 'least', 'since', 'anyone', 'once', 'can', 'bill', 'hereafter', 'be', 'seems', 'their', 'myself', 'nine', 'also', 'system', 'at', 'more', 'out', 'twelve', 'therein', 'almost', 'except', 'last', 'did', 'something', 'besides', 'via', 'whenever', 'formerly', 'cry', 'one', 'hundred', 'sixty', 'after', 'well', 'them', 'namely', 'empty', 'three', 'even', 'along', 'because', 'ourselves', 'such', 'top', 'due', 'inc', 'themselves', 'ii', 'th', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i','j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'mr', 'us', 'dk', 'workthis', 'youve', 'ive', 'sheve', 'heve', 'weve', 'theyve', 'didnt', 'dont', 'wouldnt', 'couldnt', 'cant', 'shouldnt', 'gl','cm', '________________________________', 'wwwultimatewritercom', 'svd', 'psg', 'zfxpsg', 'editorairsoftpresscom', 'iihf', 'miif','cae']
for word in extraWords:
    stopwords.add(word)

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KimYiuLui\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
engine = create_engine('postgresql://localhost/bookisbetter?user=postgres&password=admin')
Base = declarative_base()

class Book(Base):
    __tablename__ = "Books"
    isbn = Column(String())
    asin = Column(String())
    average_rating = Column(Float())
    description = Column(String())
    authors = Column(ARRAY(JSON()))
    isbn13 = Column(String())
    publication_year = Column(Integer())
    image_url = Column(String())
    book_id = Column(Integer, primary_key=True)
    title = Column(String())
    categories =  Column(ARRAY(String()))
    
    def __init__(self, isbn, asin, average_rating, description, authors, isbn13, publication_year,image_url, book_id, title, categories):
        self.isbn = str(isbn)
        self.asin = str(asin)
        self.average_rating = average_rating
        self.description = description
        self.authors = authors
        self.isbn13 = str(isbn13)
        try:
            self.publication_year = int(float(publication_year))
        except:
            self.publication_year = None
        self.image_url = str(image_url)
        self.book_id = int(book_id)
        self.title = title
        self.categories = categories
    
    def __repr__(self):
        return '%s/%s/%s%s/%s/%s%s/%s/%s%s/%s' % (self.isbn, self.asin, self.average_rating, self.description, self.authors, self.isbn13, self.publication_year, self.image_url, self.book_id, self.title, self.categories)


class DescriptionSimilarity(Base):
    __tablename__ = "DescriptionSimilarities"
    book_id = Column(Integer, primary_key=True)
    similar_book = Column(Integer, primary_key=True)
    cosine = Column(Float())
    user_feed = Column(Integer)
    last_modified = Column(DateTime())

    def __init__(self, book_id, similar_book, cosine, user_feed):
        self.book_id = book_id
        self.similar_book = similar_book
        self.cosine = cosine
        self.user_feed = user_feed
        self.last_modified = datetime.now()

    def __repr__(self):
        return '%s/%s/%s/%s' % (self.book_id, self.similar_book, self.cosine, self.user_feed)
    
Base.metadata.create_all(engine) 
Session = sessionmaker(bind=engine)
session = Session()

In [3]:
stop = text.ENGLISH_STOP_WORDS

def remove_noise(text):
    # Make lowercase
    text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove whitespaces
    text = text.apply(lambda x: " ".join(x.strip() for x in x.split()))
    
    text = text.replace('\n', ' ')
    
    # Remove special characters
    text = text.apply(lambda x: "".join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
    
    # Remove punctuation
    text = text.str.replace('[^\w\s]', '')
    
    # Remove numbers
    text = text.str.replace('\d+', '')
    
    text = text.str.replace('\d|["]|[,]|[(]|[)]|[$]|[;]|[-]|[.]|[/]|[\\]|[?]|[!]|[+]|[[]|[]]', '')
    
    # Remove Stopwords
    text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    # Convert to string
    text = text.astype(str)
        
    return text

#function to combine title with description, so title will become a keyword as well.
def combine_title_and_description(title, desc):
    combine = desc + ". " + title
    combine = remove_noise(combine)
    return combine

def Bert(data):
    sbert_model = SentenceTransformer('bert-base-nli-mean-tokens', device='cuda')
    return sbert_model.encode(data)

def RecommendationEngine(inputDf):
    document_embeddings = 0
    del document_embeddings
    
    print("cleaning data...")
    data = inputDf
    data.columns = ["Bid","Title", "Description"]
    data.loc[data["Description"]=="", 'Description'] = data.loc[data['Description']==""]['Title']
    data["cleaned_desc"] = combine_title_and_description(data["Title"], data["Description"])
    
    print("tokenizing data...")
    token_desc = []
    for index, row in data.iterrows():
        token_desc.append(text_to_word_sequence(row["cleaned_desc"]))

    filter_token = []
    for arr in token_desc:
        temp = []
        for i in arr:
            if not i in stopwords:
                temp.append(i)
            if len(temp) >= 70:
                break
        filter_token.append(temp)  
    
    data['token_desc'] = filter_token
    del token_desc
    del filter_token
    
    print("lemmatize the tokens and create bag of word...")
    token2string = []
    columns = ['token_desc'] 
    for index, row in data.iterrows():
        words = ''
        for col in columns:
            words += ' '.join(row[col]) + ' '
        token2string.append(words)

    data['bag_of_word'] = token2string
    
    data = data.drop(['cleaned_desc'], axis=1)
    data = data.drop(['token_desc'], axis=1)

    print("train bert model with the bag of word...")

    document_embeddings = Bert(data['bag_of_word'])
    
    del token2string
    
    return cosine_similarity(document_embeddings)

In [4]:
# this function accept up to 3 different use cases 
# 1 find similarity for whoel dataset
# 2 find new similarity based on user feedback
# 3 new book with 29999 books to find similarity for it.
# to go thru 1 loop is about 5 minutes with GPU
# ["all", "missing", "improve", "new"]
def AllDataPrep():
    df = pd.read_sql('select b.book_id, b.title, b.description from public."Books" as b', engine)
    indices = []
    start = 0
    end = 30000
    for i in range(int(np.ceil(len(df)/30000))):        
        if end in df.index: 
            indices.append([start, end])
        else:
            indices.append([start, len(df)])
        start += 30000
        end += 30000
    del df

    return indices

def ImproveDataPrep():
    df = pd.read_sql("""SELECT * FROM public."DescriptionSimilarities" WHERE   last_modified >= NOW() - '21 hour'::INTERVAL""", engine)
    modified_title = pd.Series(df['book_id']).drop_duplicates() #we only need to get new recommendation for these books because there are the last modified books
    modified_title = modified_title.reset_index(drop=True)
    modified_title = list(modified_title)
    book_id_list = [] # create a list containing modiefied title and the similar book. 
    for index, row in df.iterrows():
        if row["book_id"] not in book_id_list:
            book_id_list.append(row["book_id"])
        if row["similar_book"] not in book_id_list:
            book_id_list.append(row["similar_book"])

    limit = 10000 - (len(book_id_list) % 10000) - 1
    random_id_list = pd.read_sql(f"""SELECT book_id FROM public."Books" ORDER BY RANDOM() LIMIT {limit}""" ,engine)
    random_id_list = list(random_id_list["book_id"])
    for i in book_id_list:
        if i not in random_id_list:
            random_id_list.append(i)

    if len(random_id_list) > 10000:
        engine_list = [ list(random_id_list[i:i + 10000]) for i in range(0, len(random_id_list), 10000)]
        print(len(engine_list))
    else:
        engine_list = random_id_list
    
    return engine_list, modified_title
    
    
def NewDataPrep():
    return 3

def RecommendationInitialization(useCase):
    print("preparing data...")
    if useCase == "All":
        data = AllDataPrep()
        for i in data:
            print("batch between index: " + str(i[0]) + " and " + str(i[1]) + "...")
            df = pd.read_sql('select b.book_id, b.title, b.description from public."Books" as b', engine)[i[0]:i[1]]
            df = df.reset_index(drop=True)
            GetRecommendation(df, pd.Series(df['book_id']))
        
    elif useCase == "Update":
        data, target_id = ImproveDataPrep()
        if any(isinstance(i, list) for i in data):
            print("data: multiple batches...")
            for i in data:
                df = pd.read_sql(f"""select b.book_id, b.title, b.description from public."Books" as b where b.book_id IN {tuple(i)}""" ,engine)
                df = df.reset_index(drop=True)
                GetRecommendation(df, pd.Series(df['book_id']), target_id)
        else:
            df = pd.read_sql(f"""select b.book_id, b.title, b.description from public."Books" as b where b.book_id IN {tuple(data)}""" ,engine)
            df = df.reset_index(drop=True)
            GetRecommendation(df, pd.Series(df['book_id']), target_id)
    elif useCase == "New":
        data = NewDataPrep() 
        print(data)  
    

def GetRecommendation(data, indices, target_id=[]):
    cosine_similarity = RecommendationEngine(data)
    getTop20(indices, cosine_similarity)
    if len(target_id) == 0:
        getTop20ForAll(indices, cosine_similarity)
    else:
        getTop20ForSpecific(indices, cosine_similarity, target_id)
    
def getTop20ForAll(indices, cosine_similarity): 
    print("find similarity and inserting to database...")
    for i in indices:
        idx = indices[indices == i].index[0]
        score_series = pd.Series(cosine_similarity[idx]).sort_values(ascending = False)[1:21]

        for index, value in score_series.items():
            sim_id = indices[index].item() 
            sim_cosine = value
            qry = session.query(DescriptionSimilarity).filter(DescriptionSimilarity.book_id == i, DescriptionSimilarity.similar_book == int(sim_id)).first()
            if not qry:
                session.add(DescriptionSimilarity(book_id=i, similar_book= int(sim_id), cosine=float(sim_cosine), user_feed= int(0)))
        session.commit()

def getTop20ForSpecific(indices, cosine_similarity, target_id): 
    print("find similarity and inserting to database...")
    for i in indices:
        if i in target_id:
            idx = indices[indices == i].index[0]
            score_series = pd.Series(cosine_similarity[idx]).sort_values(ascending = False)[1:11]

            for index, value in score_series.items():
                sim_id = indices[index].item() 
                sim_cosine = value                
                qry = session.query(DescriptionSimilarity).filter(DescriptionSimilarity.book_id == i, DescriptionSimilarity.similar_book == int(sim_id)).first()
                if not qry:
                    session.add(DescriptionSimilarity(book_id=i, similar_book= int(sim_id), cosine=float(sim_cosine), user_feed= int(0)))
                else:
                    if sim_cosine > qry.cosine:
                        qry.cosine = sim_cosine
#                         qry.last_modified = datetime.now()
            session.commit()

def GetRecommendation(data, indices, target_id=[]):
    cosine_similarity = RecommendationEngine(data)
    if len(target_id) == 0:
        getTop20ForAll(indices, cosine_similarity)
    else:
        getTop20ForSpecific(indices, cosine_similarity, target_id)

In [198]:
RecommendationInitialization("Update")

preparing data...
cleaning data...
tokenizing data...
lemmatize the tokens and create bag of word...
train bert model with the bag of word...
target_id
find similarity and inserting to database...
